# Import modules

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [63]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [68]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [69]:
df = pj.convert_to_df()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 44 columns):
acct_type             14337 non-null object
approx_payout_date    14337 non-null datetime64[ns]
body_length           14337 non-null int64
channels              14337 non-null int64
country               14256 non-null object
currency              14337 non-null object
delivery_method       14321 non-null float64
description           14337 non-null object
email_domain          14337 non-null object
event_created         14337 non-null datetime64[ns]
event_end             14337 non-null datetime64[ns]
event_published       14238 non-null datetime64[ns]
event_start           14337 non-null datetime64[ns]
fb_published          14337 non-null int64
gts                   14337 non-null float64
has_analytics         14337 non-null int64
has_header            8928 non-null float64
has_logo              14337 non-null int64
listed                14337 non-null object
name              

In [90]:
'''
0s or 1s :
'show_map', 'fb_published', 'has_logo', , 'has_analytics'


integers or floats:
'gts', 'sale_duration', 'org_twitter', 'org_facebook'

'y' or 'n' :
'listed'
yn_map = {'y': 1, 'n' : 0}
'''

# bin clean nan:'has_header'

bin_dict = {0 : False, 1 : True}
nan_bin_dict = {0 : 0, 1 : 1, None:-1}
yn_map = {'y': 1, 'n' : 0}


# bin_cols = ['show_map', 'fb_published', 'has_logo', 'has_analytics']
# for col in bin_cols:
#     df[col] = df[col].map(bin_dict)

# yn_cols = ['listed']
# for col in yn_cols:
#     df[col] = df[col].map(yn_map)

bin_nan_cols = ['has_header']
for col in bin_nan_cols:
    df[col] =
    df[col] = df[col].map(nan_bin_dict)
    

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 44 columns):
acct_type             14337 non-null object
approx_payout_date    14337 non-null datetime64[ns]
body_length           14337 non-null int64
channels              14337 non-null int64
country               14256 non-null object
currency              14337 non-null object
delivery_method       14321 non-null float64
description           14337 non-null object
email_domain          14337 non-null object
event_created         14337 non-null datetime64[ns]
event_end             14337 non-null datetime64[ns]
event_published       14238 non-null datetime64[ns]
event_start           14337 non-null datetime64[ns]
fb_published          14337 non-null object
gts                   14337 non-null float64
has_analytics         14337 non-null object
has_header            8928 non-null float64
has_logo              14337 non-null object
listed                14337 non-null int64
name            